pyESM WORKFLOW

In [ ]:
# create an instance of Interface class
# based on the model settings, this generates the model's input files to be filled
# and the blank SQL database

from src.frontend.interface import Interface

test = Interface(
    log_level='debug',
    log_format='minimal',
    log_file_name='model_test.log',
)

In [ ]:
# load model sets from the xlsx files filled by the user
# (data validation in excel according to the constant file is envisaged)
test.model.database.load_sets()

In [ ]:
# once sets has compiled in excel and loaded in the database, variables 
# coordinates are loaded and blank database is generated
test.model.database.generate_blank_database()

In [ ]:
# generate input files to filled by the user
test.model.database.generate_input_files()

In [ ]:
# load input file/s data into the sql database
test.model.database.load_input_files()

In [1]:
# use this to start from existing database
from src.frontend.interface import Interface

test = Interface(
    log_level='debug',
    log_format='minimal',
    log_file_name='model_test.log',
)

test.warm_start_for_debug()

INFO | Interface | 'Interface' object initialization...
INFO | Interface.file_manager | 'FileManager' object generated.
DEBUG | Interface.file_manager | File 'settings.yml' loaded.
INFO | Interface.model | 'Model' object initialization...
WARNING | Interface.file_manager | Directory 'test_case_1' already exists.
WARNING | Interface.file_manager | Directory 'test_case_1' not overwritten.
INFO | Interface.model.database | 'Database' object initialization...
INFO | Interface.model.database.sql_manager | 'SQLManager' object generated.
INFO | Interface.model.database | 'Database' object initialized.
INFO | Interface.model.problem | 'Problem' object initialization...
INFO | Interface.model.problem | 'Problem' object initialized.
INFO | Interface.model | 'Model' initialized.
INFO | Interface | 'Interface' object initialized.
DEBUG | Interface.model.database.sql_manager | Connection to 'database.db' opened.
DEBUG | Interface.file_manager | Excel file 'sets.xlsx' loaded.
DEBUG | Interface.model

In [2]:
test.model.variables_generation()
test.model.variables


INFO | Interface.model.database | Generating empty dictionary of variables data...
INFO | Interface.model.database | Empty dictionary of variables data generated.


{'v': {'Reference': {2023: None, 2024: None}},
 'u': {'Reference': {2023: None, 2024: None}},
 'Y': {'Reference': {2023: None, 2024: None}},
 'Q': {'Reference': {2023: None, 2024: None}},
 'X': {'Reference': {2023: None, 2024: None}}}

In [11]:
test.model.database.coordinates

{'v': {'s_Name': ['Reference'],
  'dt_Name': [2023, 2024],
  't_Name': ['Gas power plant', 'PV power plant', 'Steel factory'],
  'f_Name': ['Energy, gas', 'Steel', 'Energy, PV']},
 'u': {'s_Name': ['Reference'],
  'dt_Name': [2023, 2024],
  'f_Name': ['Energy', 'Steel'],
  't_Name': ['Gas power plant', 'PV power plant', 'Steel factory']},
 'Y': {'s_Name': ['Reference'],
  'dt_Name': [2023, 2024],
  'f_Name': ['Energy', 'Steel'],
  't_Name': ['Final demand']},
 'Q': {'s_Name': ['Reference'],
  'dt_Name': [2023, 2024],
  'f_Name': ['Energy', 'Steel']},
 'X': {'s_Name': ['Reference'],
  'dt_Name': [2023, 2024],
  't_Name': ['Gas power plant', 'PV power plant', 'Steel factory']}}

In [21]:
from src.util.util import prettify
prettify(test.model.database.variables_info['v'])

{'coordinates': {'datetime': 'all',
                 'flows': {'set_categories': 'f.p'},
                 'scenarios': 'all',
                 'technologies': {'set_categories': 't.s'}},
 'name': 'make coefficients matrix',
 'set_headers': 'name',
 'shape': ['technologies', 'flows'],
 'symbol': 'v',
 'type': 'exogenous',
 'var_dict_hierarchy': None}


In [ ]:
test.model.database.sets_structure

In [ ]:
test.model.database.sets

In [10]:
test.model.database.sqltools.open_connection()

var = test.model.database.sqltools.filtered_table_to_dataframe(
    table_name='v',
    filters_dict={
        's_Name': ['Reference'],
        'dt_Name': [2023],
        't_Name': ['Gas power plant'],
        'f_Name': ['Steel']
    }
)

filter_1 = test.model.database.sqltools.get_related_table_keys(
    parent_table_name='_set_FLOWS',
    child_column_name='f_Name',
    parent_table_fields={
        'f_Category': ['Product flow'],
    }
)

filter_2 = test.model.database.sqltools.get_related_table_keys(
    parent_table_name='_set_DATETIME',
    child_column_name='dt_Name',
    parent_table_fields={
        'dt_Name': [2023],
    }
)

var_custom_filter = test.model.database.sqltools.filtered_table_to_dataframe(
    table_name='v',
    filters_dict={
        **filter_1,
        **filter_2,
        **{'f_Name': ['Steel']},
    }
)

test.model.database.sqltools.close_connection()

# filter_1, filter_2
var_custom_filter

DEBUG | Interface.model.database.sql_manager | Connection to 'database.db' opened.
DEBUG | Interface.model.database.sql_manager | Connection to 'database.db' closed.


,id,s_Name,dt_Name,t_Name,f_Name,values
0,0,Reference,2023,Gas power plant,Steel,1.0
1,3,Reference,2023,PV power plant,Steel,4.0
2,6,Reference,2023,Steel factory,Steel,7.0


In [ ]:
# delete all the information included in the model data folder, if necessary
test.model.model_dir_cleanup()